# Import datase & Library

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transaction = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/transaction.csv', parse_dates=['transaction_created_datetime', 'transaction_updated_datetime'])
user = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/user.csv')
promotion = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/promotion.csv')

# Dataset info

In [ ]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   dpt_id                        50000 non-null  object        
 1   dpt_promotion_id              50000 non-null  object        
 2   buyer_id                      50000 non-null  object        
 3   seller_id                     50000 non-null  object        
 4   transaction_amount            50000 non-null  float64       
 5   payment_method_name           50000 non-null  object        
 6   payment_provider_name         50000 non-null  object        
 7   transaction_created_datetime  50000 non-null  datetime64[ns]
 8   transaction_updated_datetime  50000 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 3.4+ MB


In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9527 entries, 0 to 9526
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   company_id                   9527 non-null   object 
 1   company_kyc_status_name      9527 non-null   object 
 2   company_kyb_status_name      9527 non-null   object 
 3   company_type_group           9527 non-null   object 
 4   company_phone_verified_flag  9527 non-null   float64
 5   company_email_verified_flag  9527 non-null   float64
 6   user_fraud_flag              9527 non-null   float64
 7   testing_account_flag         9527 non-null   float64
 8   blacklist_account_flag       9527 non-null   float64
 9   package_active_name          9527 non-null   object 
 10  company_registered_datetime  9527 non-null   object 
dtypes: float64(5), object(6)
memory usage: 818.9+ KB


In [ ]:
promotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dpt_promotion_id                   729 non-null    object 
 1   promotion_code                     729 non-null    object 
 2   promotion_name                     729 non-null    object 
 3   transaction_promo_cashback_amount  729 non-null    float64
dtypes: float64(1), object(3)
memory usage: 22.9+ KB


# Buyer-Seller Relationship Score

### cek distribusi data

In [ ]:
# 1. Hitung total transaksi untuk setiap pasangan buyer-seller
buyer_seller_transactions = transaction.groupby(['buyer_id', 'seller_id'])['dpt_id'].count().reset_index()
buyer_seller_transactions.rename(columns={'dpt_id': 'transaction_count'}, inplace=True)

In [ ]:
print(f"skewness: {buyer_seller_transactions['transaction_count'].skew()}")
print(f"kurtosis: {buyer_seller_transactions['transaction_count'].kurtosis()}")

skewness: 40.28050120483677
kurtosis: 2647.034188503297


Skewness sebesar 40 menunjukkan bahwa distribusi 'transaction_count' sangat miring (highly skewed) dan dipengaruhi secara signifikan oleh outlier. Nilai kurtosis sebesar 2647 mengonfirmasi keberadaan outlier yang sangat ekstrem, yang memengaruhi distribusi data secara signifikan. Oleh karena itu, transformasi logaritmik dan normalisasi dilakukan sebelum membuat relationship_score

### membuat kolom relationship_score

In [ ]:
# 2. Transformasi logaritmik untuk mengurangi skewness
buyer_seller_transactions['log_transaction_count'] = np.log1p(buyer_seller_transactions['transaction_count'])

# 3. Normalisasi ke rentang 1-5   sangat lemah, lemah, sedang, kuat, sangat kuat
min_log = buyer_seller_transactions['log_transaction_count'].min()
max_log = buyer_seller_transactions['log_transaction_count'].max()
buyer_seller_transactions['relationship_score'] = (
    1 + 4 * (buyer_seller_transactions['log_transaction_count'] - min_log) / (max_log - min_log)
).round().astype(int)

# Hapus kolom tidak diperlukan
buyer_seller_transactions.drop(columns=['transaction_count', 'log_transaction_count'], inplace=True)

In [ ]:
buyer_seller_transactions

,buyer_id,seller_id,relationship_score
0,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,1
1,001046b5061e28476b83fe2335b04d3210bed72a2fee17...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
2,00119737eef11ff1d30c2061dd1e19c06d963d5a125c92...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
3,0012614e5a1366f102a3497b67f8ec9a8009c802aa6959...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
4,0013cdaff46e67574660e0ddd214e5032e3ff5d94744e1...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2
...,...,...,...
10349,fff5a90a971c4b76a32653314e4b344d3d492f197837db...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2
10350,fff5f7cf3deff1c2acad4533c4b845f8f2c0c05168f668...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
10351,fff77b856ac2478f8911b9cddf65980ac7fdc4d700e354...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
10352,fff9528eb5496b60fd3e824c66154f2d2f06c27c172a4a...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1


# Transaction Frequency Metrics

Peninjauan burst activity, unusual gap, burst_amount dilakukan terhadap user yang telah melakukan transaksi minimal 5 kali

In [ ]:
trans_freq_metrics = transaction[['dpt_id', 'buyer_id', 'seller_id', 'transaction_created_datetime', 'transaction_amount']].copy()

# Konversi ke datetime dan urutkan data
trans_freq_metrics['transaction_created_datetime'] = pd.to_datetime(trans_freq_metrics['transaction_created_datetime'])
trans_freq_metrics = trans_freq_metrics.sort_values(by=['buyer_id', 'transaction_created_datetime'])

# Hitung time_diff untuk setiap buyer
trans_freq_metrics['time_diff'] = trans_freq_metrics.groupby('buyer_id')['transaction_created_datetime'].diff().dt.total_seconds().fillna(0)

# Filter hanya buyer dengan minimal 5 transaksi
trans_freq_metrics['transaction_count'] = trans_freq_metrics.groupby('buyer_id')['transaction_created_datetime'].transform('count')

# Menghitung median dan IQR untuk time_diff (hanya untuk buyer dengan transaksi >= 5)
trans_freq_metrics['median_time_diff'] = trans_freq_metrics.groupby('buyer_id')['time_diff'].transform(lambda x: x.median() if len(x) >= 5 else 0)
trans_freq_metrics['iqr_time_diff'] = trans_freq_metrics.groupby('buyer_id')['time_diff'].transform(lambda x: x.quantile(0.75) - x.quantile(0.25) if len(x) >= 5 else 0)

# Definisi threshold burst dan gap menggunakan median/IQR untuk time_diff
BURST_THRESHOLD_RATIO = 0.1   # 10% dari median
GAP_MULTIPLIER = 1.5           # 1.5x IQR

# Fitur burst_activity: transaksi dengan time_diff kecil dibandingkan median dan time_diff > 0
trans_freq_metrics['burst_activity'] = np.where(
    (trans_freq_metrics['transaction_count'] >= 5) &
    (trans_freq_metrics['time_diff'] > 0) &
    (trans_freq_metrics['time_diff'] < BURST_THRESHOLD_RATIO * trans_freq_metrics['median_time_diff']),
    1, 0
)

# Fitur unusual_gap: transaksi dengan time_diff jauh di atas median + IQR dan time_diff > 0
trans_freq_metrics['unusual_gap'] = np.where(
    (trans_freq_metrics['transaction_count'] >= 5) &
    (trans_freq_metrics['time_diff'] > trans_freq_metrics['median_time_diff'] + GAP_MULTIPLIER * trans_freq_metrics['iqr_time_diff']),
    1, 0
)

# Fitur burst_amount: transaksi yang jauh lebih besar/kecil dari data historis buyer
trans_freq_metrics['median_amount'] = trans_freq_metrics.groupby('buyer_id')['transaction_amount'].transform('median')
trans_freq_metrics['iqr_amount'] = trans_freq_metrics.groupby('buyer_id')['transaction_amount'].transform(lambda x: x.quantile(0.75) - x.quantile(0.25))

# Definisi threshold untuk burst_amount
BURST_AMOUNT_MULTIPLIER = 1.5  # Faktor untuk IQR
trans_freq_metrics['burst_amount'] = np.where(
    (trans_freq_metrics['transaction_count'] >= 5) &
    (
        (trans_freq_metrics['transaction_amount'] > trans_freq_metrics['median_amount'] + BURST_AMOUNT_MULTIPLIER * trans_freq_metrics['iqr_amount']) |
        (trans_freq_metrics['transaction_amount'] < trans_freq_metrics['median_amount'] - BURST_AMOUNT_MULTIPLIER * trans_freq_metrics['iqr_amount'])
    ),
    1, 0
)

# Set nilai 0 untuk buyer dengan transaksi < 5
trans_freq_metrics['burst_activity'] = np.where(trans_freq_metrics['transaction_count'] < 5, 0, trans_freq_metrics['burst_activity'])
trans_freq_metrics['unusual_gap'] = np.where(trans_freq_metrics['transaction_count'] < 5, 0, trans_freq_metrics['unusual_gap'])
trans_freq_metrics['burst_amount'] = np.where(trans_freq_metrics['transaction_count'] < 5, 0, trans_freq_metrics['burst_amount'])

# Konversi hasil ke integer
trans_freq_metrics['burst_activity'] = trans_freq_metrics['burst_activity'].astype(int)
trans_freq_metrics['unusual_gap'] = trans_freq_metrics['unusual_gap'].astype(int)
trans_freq_metrics['burst_amount'] = trans_freq_metrics['burst_amount'].astype(int)

# Hapus kolom tambahan jika tidak diperlukan
trans_freq_metrics = trans_freq_metrics.drop(columns=['transaction_count', 'median_time_diff', 'iqr_time_diff', 'median_amount', 'iqr_amount'])


In [ ]:
trans_freq_metrics

,dpt_id,buyer_id,seller_id,transaction_created_datetime,transaction_amount,time_diff,burst_activity,unusual_gap,burst_amount
22038,befdc59cf11ec39defab73c7baa0bd1c16ea7e003b59af...,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,2023-12-27 08:47:56.463138,1.007000e+05,0.000000e+00,0,0,0
20987,2bc105494f6030d5194fc6fdc50d9acede7d122dd80b3a...,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,2023-12-28 11:30:42.785083,1.510500e+05,9.616632e+04,0,0,0
30963,d8c359cc9de4f730939cfa6c5b50bac2215cf994a30816...,001046b5061e28476b83fe2335b04d3210bed72a2fee17...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-08-13 11:50:37.122442,2.000008e+08,0.000000e+00,0,0,0
9971,769c7fdcc0e36175c9363c6d822ebb5b9c8748acb8c12e...,001046b5061e28476b83fe2335b04d3210bed72a2fee17...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-12-29 19:10:25.077279,3.000000e+08,1.194959e+07,0,0,0
21264,d808c4f7daf19127c77a79dffadf947d9a5d4fa4a9ecb7...,00119737eef11ff1d30c2061dd1e19c06d963d5a125c92...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-05-05 10:12:38.083254,2.029000e+07,0.000000e+00,0,0,0
...,...,...,...,...,...,...,...,...,...
9049,3220d105503b34c8a43da87b7ac5365923742d74e699ae...,fff5f7cf3deff1c2acad4533c4b845f8f2c0c05168f668...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-11-27 13:27:35.604107,1.015000e+07,0.000000e+00,0,0,0
25856,0785c548ad9260f4c65b37836520e3781c9fd1cd441431...,fff77b856ac2478f8911b9cddf65980ac7fdc4d700e354...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-11-30 18:00:45.414410,2.826775e+06,0.000000e+00,0,0,0
17692,7862efbfd8e68bd315332118e2c1dfb7e825ceb06819b0...,fff9528eb5496b60fd3e824c66154f2d2f06c27c172a4a...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-06-15 09:26:40.813787,2.500000e+07,0.000000e+00,0,0,0
24571,af0c97c859e45bcc17205bc081a7cfaa73c19a31369e22...,fff9528eb5496b60fd3e824c66154f2d2f06c27c172a4a...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,2023-07-15 15:02:32.964113,5.062312e+07,2.612152e+06,0,0,0


# Promotion Exploitation Indicator

In [ ]:
# Step 1: Hitung frekuensi penggunaan promo untuk setiap company_id dan dpt_promotion_id
promo_usage = transaction.groupby(['buyer_id', 'dpt_promotion_id'])['transaction_created_datetime'].count().reset_index()

# Rename kolom
promo_usage = promo_usage.rename(columns={
    'buyer_id': 'company_id',             # Ganti buyer_id menjadi company_id
    'transaction_created_datetime': 'promo_usage_count'  # Ganti nama kolom hasil agregasi
})

# Step 2: Tetapkan promo_usage_count menjadi 0 jika dpt_promotion_id == 'no promotion'
promo_usage.loc[promo_usage['dpt_promotion_id'] == 'no promotion', 'promo_usage_count'] = 0

# Step 3: Tentukan threshold untuk penyalahgunaan promo
PROMO_USAGE_THRESHOLD = 3
promo_usage['is_promotion_exploitation'] = np.where(
    promo_usage['promo_usage_count'] > PROMO_USAGE_THRESHOLD, 1, 0
)

# Step 4: Jika ada is_promotion_exploitation == 1 untuk company_id, setel menjadi 1
promotion_exploitation = promo_usage.groupby(['company_id'])['is_promotion_exploitation'].max().reset_index()

In [ ]:
# Final dataset: hanya dua kolom company_id dan is_promotion_exploitation
promotion_exploitation.sort_values(by='is_promotion_exploitation', ascending=False)


,company_id,is_promotion_exploitation
1808,3077819ec94241590c88a38ed75fa3ef4bbe5a3328d805...,1
602,0fe36baa803c6718b95994af1e4a31cf1cbce48c09a623...,1
7395,c799c01d50a17e52ffa074b3a6bad0f50781c6daad7d38...,1
0,00048ebf5503ef1dfb03eec05312583eeb2b57320ac682...,0
6354,ab6bfa604c13030538d51479787a32e63ba05496e2844b...,0
...,...,...
3177,5618695192a7c0768749358b822dcbd8b606567bd6fa54...,0
3178,5623b4dec937d89e8cdf70186b96d865d23b0eeefe9a7b...,0
3179,562bc508cbc8ef045114f5a95ad3774a745dd0fa25d3f7...,0
3180,562f83390b31e03ee80ff50d94e99197a6e14c8ccea206...,0


# Scaling & Normalization

### Normalisasi transaction_amount

In [ ]:
# Cek distribusi transaction_amount
print(f"skewness: {transaction['transaction_amount'].skew()}")
print(f"kurtosis: {transaction['transaction_amount'].kurtosis()}")

skewness: 139.0426276754273
kurtosis: 26266.25394224923


In [ ]:
transaction['log_transaction_amount'] = np.log1p(transaction['transaction_amount'])
# Normalize transaction_amount
transaction['normalized_transaction_amount'] = MinMaxScaler().fit_transform(
    transaction[['log_transaction_amount']]
)

### Normalisasi transaction_created_datetime

In [ ]:
# Convert transaction_created_datetime to epoch timestamp
transaction['transaction_created_datetime'] = pd.to_datetime(
    transaction['transaction_created_datetime']
)
transaction['transaction_created_timestamp'] = transaction[
    'transaction_created_datetime'
].astype(np.int64) // 10**9

print(f"skewness: {transaction['transaction_created_timestamp'].skew()}")
print(f"kurtosis: {transaction['transaction_created_timestamp'].kurtosis()}")

skewness: -0.23816982888219002
kurtosis: -0.9343853344805999


In [ ]:
# Normalize transaction_created_timestamp
transaction['normalized_transaction_time'] = MinMaxScaler().fit_transform(
    transaction[['transaction_created_timestamp']]
)
transaction.drop(columns=['log_transaction_amount', 'transaction_created_timestamp'], inplace=True)

In [ ]:
transaction

,dpt_id,dpt_promotion_id,buyer_id,seller_id,transaction_amount,payment_method_name,payment_provider_name,transaction_created_datetime,transaction_updated_datetime,normalized_transaction_amount,normalized_transaction_time
0,69e9566b3f4d6cb7db2216121b0cd0308c516e680e9c78...,no promotion,bbce610a3267808752a7ec263a7ecfbe76a4987d529bcb...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,20380.0,MITRA_PEMBAYARAN_DIGITAL,BLIBLI,2023-08-16 09:00:53.297729,2023-08-16 10:24:56.875352,0.418206,0.801853
1,961d6f7efde3622da8f35e76e2b53acd84c647a661de86...,no promotion,09eb3b80abae1238ef39d50b66215e02e1ac9891ad6e8f...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,14673.6,MITRA_PEMBAYARAN_DIGITAL,BLIBLI,2023-06-09 15:22:49.867524,2023-06-09 15:23:42.718184,0.404362,0.703608
2,6441defc089b4ae947cce529904a5c7db2326ede52bdaa...,no promotion,25d0774533d69564d0deca724a55a76c693ed5f7ffa12a...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1012500.0,MITRA_PEMBAYARAN_DIGITAL,TOKOPEDIA,2023-10-08 10:45:24.139583,2023-10-09 11:47:23.938359,0.582818,0.878832
3,64152dd86b2c5d1af6aaa911e7a229dc539273e0a7b8be...,no promotion,5b846313375cb4f4d065e50a05833dc3ac20ba3f532bbe...,5b846313375cb4f4d065e50a05833dc3ac20ba3f532bbe...,30000.0,MITRA_PEMBAYARAN_DIGITAL,BLIBLI,2023-05-26 13:41:27.133014,2023-05-26 13:47:40.595121,0.434502,0.683199
4,ae4ddde99c8fe7f73fb3c2ee0e157e886b52417ece17bd...,no promotion,5c19a13a9b229340b584f621b648f4dec7491e12368392...,5c19a13a9b229340b584f621b648f4dec7491e12368392...,1000008.0,MITRA_PEMBAYARAN_DIGITAL,TOKOPEDIA,2023-05-26 17:29:34.201724,2023-05-26 17:30:04.688498,0.582295,0.683429
...,...,...,...,...,...,...,...,...,...,...,...
49995,4da123d84a819edf166f67a9cc0c197f81967aa9717ed5...,no promotion,5edae69c82a6ef7c97288ea207d4cf9193997657c93d22...,5edae69c82a6ef7c97288ea207d4cf9193997657c93d22...,1000000.0,MITRA_PEMBAYARAN_DIGITAL,TOKOPEDIA,2023-06-19 13:52:12.117180,2023-06-19 13:53:09.785285,0.582295,0.718021
49996,0b358a09e15d062e1d9e0fafb8a986d802c2044b150a6e...,no promotion,f83e98fe775227a355ded830adb7c59914f2bcfd32965c...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,50039500.0,MITRA_PEMBAYARAN_DIGITAL,TOKOPEDIA_CREDIT_CARD,2023-06-02 11:53:06.595470,2023-06-02 11:56:42.181481,0.747211,0.693243
49997,ba67db08614a504cc5c9042cc61c914fa82748d526f0f3...,no promotion,02611e2fdd7d730bddbd654baf24f03a739704bcb34c01...,02611e2fdd7d730bddbd654baf24f03a739704bcb34c01...,4531500.0,QRIS,QRIS,2023-10-23 14:06:05.090016,2023-10-23 14:06:05.090016,0.645982,0.900790
49998,351da251d731cba588418e4a175460cc822ecbeb751751...,no promotion,a55674077f7db3b4ecb2ba1805e58c966469e00a532dfb...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,8424500.0,CREDIT_CARD,MASTERCARD,2023-11-05 16:24:20.163636,2023-11-05 16:25:34.101821,0.672118,0.919785


# Simpan file hasil feature engineering ke direktori Google Drive

In [ ]:
# Simpan file CSV ke direktori Google Drive
buyer_seller_transactions.to_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/feature_engineering/relationship_score.csv', index=False)
trans_freq_metrics.to_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/feature_engineering/transaction_frequency_metrics.csv', index=False)
promotion_exploitation.to_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/feature_engineering/promotion_exploitation.csv', index=False)
transaction.to_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/feature_engineering/transaction_scaled.csv', index=False)